In [5]:
### Step 1: Initiation ###

from rdkit import Chem, rdBase
from rdkit.Chem import Draw,rdDepictor
from rdkit.Chem import AllChem, Descriptors, DataStructs
from rdkit.Chem.Draw import IPythonConsole, rdMolDraw2D
from rdkit.Chem.Scaffolds import rdScaffoldNetwork
from datetime import datetime
# import math
# import matplotlib as mpl
# from matplotlib import pyplot as plt
# import os
# import pyvis
# from pyvis.network import Network
# import inspect
import psycopg2
from collections import defaultdict

# DATABASE Connections

# Chemistry Database
#db_chem = psycopg2.connect(host = "localhost", dbname="Chemistry", user="postgres", password="postgres")
db_chem = psycopg2.connect(host = "192.168.86.31", dbname="Chemistry", user="postgres", password="postgres")
# create a cursor
cur = db_chem.cursor()
# execute a statement
cur.execute('SELECT version()')
db_ver = cur.fetchone()
print('PostgreSQL database version:', db_ver) 
#display the PostgreSQL database server version


print('RDKit version: ',rdBase.rdkitVersion)
# print('MatplotLib version:', mpl.__version__)
# print('Pyvis version:', pyvis.__version__)
print(datetime.now())

###################################################################################################################

### Step 2: Generate Fingerprints for New Structures ("no fingerprints") ###

import collections
import fingerprinter as fpt

# SELECT Structures without fingerprints

# db_chem = psycopg2.connect(host = "localhost", dbname="Chemistry", user="postgres", password="postgres")
cur = db_chem.cursor() 
sql = 'SELECT s."Structure_ID", s."SMILES" FROM public."Structures" s \
WHERE s."Structure_ID" IN (SELECT f."Structure_ID" FROM public."Structures" f \
WHERE NOT EXISTS (SELECT 1 FROM public."Structure_Fingerprint" fp WHERE f."Structure_ID" = fp."Structure_ID"))' 
cur.execute(sql) 

mollist = cur.fetchall()
print()
print('##### Structures without Fingerprints #####')
print()
print(len(mollist),'new Structures')
print(mollist)


# generate Morgan fingerprints for molecules

fp_key = {}
fp_val = []
fp_dict = collections.defaultdict(list)
fp_allmol = []
fp_smi_list = {}

for mol in mollist:
    print()
    print('Molecule : ',mol)
    # print(mol[0], mol[1])
    m = Chem.MolFromSmiles(mol[1])

    # Generate Fingerprint SMILES and SMILES list
    fp_smi = fpt.FingerprintToSmiles(m, 3)
    print('Fingerprint : ',fp_smi)    
    for f in fp_smi: 
        fp_smi_list[f[0]] = f[1]
             
    # Generate Fingerprint Footprints:
    big ={}
    fp = AllChem.GetMorganFingerprint(m,3, bitInfo=big)
    # print(len(big))
    # print(big)
    # print(fp)
    for fpb,v in big.items():
        ma = []
        fp_dict[fpb].append(len(v))
        # print(mol[0],fpb, len(v),v)
        ma.append(mol[0])
        ma.append(fpb)
        ma.append(len(v))
        # print(ma)
        fp_allmol.append(ma)
        
print()
print("All Fingerprints by molecule")
print(fp_allmol)
        
print()        
print(fp_smi_list)
print(len(fp_smi_list),'SMILES')

for fpb,v in fp_smi_list.items():
    
    print(fpb,' - ',v)
    
        # key = fbp
        #fp_dict[fpb].append(len(v))
                
print()        
# print(fp_dict)
print(len(fp_dict),'fingerprints')

for fpb,v in fp_dict.items():
    print(fpb,' - ',len(v),' - ',v)
        # key = fbp
        #fp_dict[fpb].append(len(v))
        
###################################################################################################################

### Step 3: Insert new fingerprints into fingerprint table ###

print()
print('##### Insert New Fingerprints #####')
print()

# db_chem = psycopg2.connect(host = "localhost", dbname="Chemistry", user="postgres", password="postgres")
cur = db_chem.cursor()
sql = 'SELECT * FROM public."Fingerprints" WHERE "FP_Type" = 1'
cur.execute(sql)
fpdblist = cur.fetchall()
# print(len(fpdblist), "existing fingerprints")
# print(fpdblist)
# print(fp_smi_list)

print()
if len(fp_smi_list) == 0:
    print('No new fingerprint inserts')
else:
    print("New inserted fingerprints")
print()

sql = 'INSERT INTO public."Fingerprints"("FP_Smiles", "FP_Number", "FP_Type") VALUES(%s, %s, 1);'

for fpb,v in fp_smi_list.items():
    smi_fl = 0
    for i in range(0,len(fpdblist)):
        if fpb == fpdblist[i][3]:
            smi_fl = 1

    if smi_fl == 0:
        print(fpb,' - ',v, smi_fl, "inserted")
        cur.execute(sql, (v, fpb))
        db_chem.commit()
        #print(fpb,' - ',v, smi_fl, "inserted")

###################################################################################################################

### Step 4: Insert Fingerprints for New Molecules ###

print()
print('##### Insert Structure/Fingerprints Links #####')
print()

# db_chem = psycopg2.connect(host = "localhost", dbname="Chemistry", user="postgres", password="postgres")
cur = db_chem.cursor()
sql = 'SELECT * FROM public."Fingerprints" WHERE "FP_Type" = 1'
cur.execute(sql)
fpdblist = cur.fetchall()
# print(fpdblist)

fpid = {}

for m in fpdblist:
    # print(m)
    i = m[0]
    j = m[3]
    # print(i,j)
    fpid[j]=i
    
# print(fpid)

cur = db_chem.cursor()
sql = 'INSERT INTO public."Structure_Fingerprint"("Structure_ID","FP_ID","FP_Count") VALUES (%s,%s,%s);'

# Insert Notification
print()
if len(fp_allmol) == 0:
    print('No new structure/fingerprint inserts')
else:
    print('New structure/fingerprint links')
print()

for m in fp_allmol:
    # j = m[1]
    i = fpid[m[1]]
    cur.execute(sql, (m[0],i,m[2]))
    db_chem.commit()
    print(m,i)
    
###################################################################################################################

### Step 5: Update Shannon Entropies ###
print()
print('##### Shannon Update #####')
print()

# This is part of netprepper
#    import pandas as pd
#    from scipy import stats as scistat

# import sys
# print(sys.path)
# set PYTHONPATH to proper directory set holds private modules and packages (here pkg_mod)
import netprepper

# SELECT fingerprint length and append to fp_dict

#db_chem = psycopg2.connect(host = "localhost", dbname="Chemistry", user="postgres", password="postgres")
db_chem = psycopg2.connect(host = "192.168.86.31", dbname="Chemistry", user="postgres", password="postgres")
cur = db_chem.cursor() 
sql = 'SELECT sf."Structure_ID", sf."FP_ID", sf."FP_Count", fp."FP_Shannon" FROM public."Structure_Fingerprint" sf \
JOIN public."Fingerprints" fp ON sf."FP_ID" = fp."Fingerprint_ID" WHERE fp."FP_Type" = 1'
cur.execute(sql)
fp_list = cur.fetchall()
# print(fp_list)

fp_dict = collections.defaultdict(list)
fp_mol = collections.defaultdict(list)

for m in fp_list:
    # print(m, m[1],m[2])
    fp_dict[m[1]].append(m[2])
    fp_mol[m[0]].append(m[1])

# n_fp = 0
print(len(fp_dict),'fingerprints')
print(len(fp_mol),'structures')
print()

n_fp = len(fp_mol)

#for fpb, i in fp_dict.items():
#    n_fp = max(n_fp, len(i))
    # print (fpb, i, n)


# update entropy field in Fingerprints    
    
sql = 'UPDATE public."Fingerprints" SET "FP_Shannon" = %s WHERE "Fingerprint_ID" = %s'
cur = db_chem.cursor()
for fpb, i in fp_dict.items():
    ns = netprepper.shannon(i,n_fp)
    # print(fpb,' - ',netprepper.shannon(i,n))
    print(fpb,' - ',ns[0],' - ',ns[1])
    cur.execute(sql, (ns[1], fpb))
db_chem.commit()



PostgreSQL database version: ('PostgreSQL 12.7 (Ubuntu 12.7-1.pgdg20.04+1) on x86_64-pc-linux-gnu, compiled by gcc (Ubuntu 9.3.0-17ubuntu1~20.04) 9.3.0, 64-bit',)
RDKit version:  2020.03.3
2021-07-26 23:28:46.460774

##### Structures without Fingerprints #####

7 new Structures
[(49, 'C1(=C2C(=NC=N1)[N](C=N2)[C@@H]3[C@@H]([C@H](CO3)O)O)O'), (50, 'C1=NC(=C2C(=N1)N(C=N2)[C@H]3[C@@H]([C@@H]([C@H](O3)COP(=O)(O)OP(=O)(O)OP(=O)(O)O)O)O)N'), (51, 'NCCOCCCC(=O)N[C@@H](CC1=CC=C(OP(O)(=O)NP(O)(=O)OP(O)(=O)OC[C@H]2O[C@H]([C@H](O)[C@@H]2O)N2C=NC3=C(N)N=CN=C23)C=C1)C(O)=O'), (56, 'C1=CC(=CC=C1C=CC(=O)C2=C(C=C(C=C2)O)O)O'), (57, 'C1=CC(=CC=C1/C=C/C(=S)C2=C(C=C(C=C2)O)O)O'), (58, 'CC(C(C1=NC=C(C=C1)O)O)C(C(=S)NC(C2C(C(C(O2)N3C=CC(=O)NC3=O)O)O)C(=O)O)N'), (59, 'C[C@H]([C@@H](C1=NC=C(C=C1)O)O)[C@@H](C(=O)N[C@@H]([C@@H]2[C@H]([C@H]([C@@H](O2)N3C=CC(=O)NC3=O)O)O)C(=O)O)N')]

Molecule :  (49, 'C1(=C2C(=NC=N1)[N](C=N2)[C@@H]3[C@@H]([C@H](CO3)O)O)O')
Fingerprint :  [(11507010, 'COC[C@@H](C)O'), (26234434, '

951226070  -  5  -  [4, 6, 6, 3, 3]
1018672370  -  1  -  [1]
1085753368  -  1  -  [1]
1139073387  -  1  -  [2]
1148601599  -  1  -  [1]
1167322652  -  1  -  [1]
1211602019  -  1  -  [1]
1238084039  -  1  -  [1]
1277979401  -  1  -  [1]
1362518133  -  3  -  [1, 1, 1]
1429883190  -  1  -  [2]
1434568782  -  1  -  [1]
1510328189  -  4  -  [2, 1, 1, 2]
1510461303  -  1  -  [1]
1511301995  -  1  -  [1]
1533864325  -  3  -  [1, 1, 1]
1599104900  -  1  -  [1]
1629008723  -  1  -  [1]
1833907439  -  1  -  [1]
2027655192  -  1  -  [1]
2134940345  -  1  -  [1]
2215059400  -  1  -  [1]
2245273601  -  3  -  [1, 4, 4]
2246699815  -  5  -  [2, 1, 1, 2, 2]
2354817108  -  1  -  [1]
2592785365  -  1  -  [1]
2833572456  -  1  -  [1]
2857747142  -  1  -  [1]
2939120473  -  1  -  [1]
2944555726  -  1  -  [2]
2966896858  -  1  -  [1]
3041753046  -  1  -  [1]
3162837314  -  1  -  [1]
3247388914  -  1  -  [1]
3279289513  -  1  -  [1]
3295942588  -  1  -  [1]
3315826729  -  1  -  [1]
3324998366  -  1  -  [1]


2575700980  -  ccc(C=CC(c)=O)cc 0 inserted
2630658760  -  ccc(C(=O)C=Cc)c(c)O 0 inserted
2693887260  -  cc(c)/C=C/C 0 inserted
2829985561  -  cC=CC(c)=O 0 inserted
3698257053  -  cc(O)cc(c)O 0 inserted
3722792302  -  ccc(C(C)=O)c(c)O 0 inserted
3988559847  -  C/C=C/c1ccccc1 0 inserted
4180801603  -  cccc(c)C 0 inserted
744747118  -  C=S 0 inserted
824737160  -  ccc(C(C)=S)c(c)O 0 inserted
877069709  -  cc(c)/C=C/C(=S)c(c)c 0 inserted
963040446  -  C=CC(=S)c1ccccc1O 0 inserted
980367002  -  C=CC(=S)c(c)c 0 inserted
1026654305  -  S 0 inserted
1463712834  -  ccc(/C=C/C(c)=S)cc 0 inserted
1780450476  -  CC(c)=S 0 inserted
3057522770  -  CC(=S)c1ccc(O)cc1O 0 inserted
3656677079  -  ccc(C(=S)/C=C/c)c(c)O 0 inserted
3868591127  -  c/C=C/C(c)=S 0 inserted
29590919  -  ccc(nc)[C@@H](O)[C@@H](C)[C@@H](C)N 0 inserted
63934028  -  CNC(=S)C(N)C(C)C(c)O 0 inserted
124907836  -  CC(C)O[C@H](C)n 0 inserted
180232782  -  CC(=S)NC(C)C 0 inserted
264951365  -  C[C@H](O)c1ccc(O)cn1 0 inserted
330223697  

[51, 2263308118, 1] 469
[51, 2300762321, 1] 434
[51, 2306146506, 1] 470
[51, 2354817108, 1] 521
[51, 2378928689, 1] 471
[51, 2428282676, 1] 472
[51, 2579617025, 1] 474
[51, 2592785365, 1] 14
[51, 2667063169, 1] 475
[51, 2698937907, 1] 436
[51, 2805248962, 1] 476
[51, 2810479046, 1] 477
[51, 2833572456, 1] 522
[51, 2849870857, 1] 478
[51, 2857747142, 1] 523
[51, 2917424199, 1] 437
[51, 2939120473, 1] 524
[51, 2944555726, 2] 525
[51, 2966896858, 1] 526
[51, 2976033787, 4] 289
[51, 2999502343, 1] 438
[51, 3041753046, 1] 527
[51, 3109000344, 1] 439
[51, 3129492592, 1] 479
[51, 3134606292, 1] 480
[51, 3162837314, 1] 528
[51, 3176806076, 1] 441
[51, 3189457552, 1] 145
[51, 3216147685, 1] 443
[51, 3217380708, 5] 42
[51, 3218693969, 6] 27
[51, 3247388914, 1] 529
[51, 3252164723, 1] 482
[51, 3279289513, 1] 530
[51, 3295942588, 1] 531
[51, 3315826729, 1] 532
[51, 3324998366, 1] 533
[51, 3345491492, 1] 483
[51, 3349586940, 1] 484
[51, 3351057854, 1] 485
[51, 3418339917, 1] 534
[51, 3496159487, 1]

16  -  [1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]  -  0.2868359830561607
17  -  [2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]  -  0.2530673318741798
18  -  [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]  -  0.12693054534140227
19  -  [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]  -  0.12693054534140227
20  -  [2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]  -  0.12693054534140227
21  -  [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]  -  0.12693054534140227
22  -  [2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]  -  0.2530673318741798
23  -  [2, 1, 1, 0, 0, 

...

355  -  [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]  -  0.2145591551764051
356  -  [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]  -  0.2145591551764051
357  -  [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]  -  0.12693054534140227
358  -  [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]  -  0.2145591551764051
359  -  [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]  -  0.2145591551764051
360  -  [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]  -  0.12693054534140227
361  -  [2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]  -  0.2145591551764051
362  -  [1, 1, 0, 

428  -  [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]  -  0.12693054534140227
429  -  [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]  -  0.12693054534140227
430  -  [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]  -  0.12693054534140227
431  -  [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]  -  0.12693054534140227
432  -  [1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]  -  0.2868359830561607
433  -  [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]  -  0.12693054534140227
434  -  [1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]  -  0.2868359830561607
435  -  [1, 0, 

499  -  [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]  -  0.12693054534140227
500  -  [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]  -  0.12693054534140227
501  -  [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]  -  0.12693054534140227
502  -  [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]  -  0.12693054534140227
503  -  [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]  -  0.12693054534140227
504  -  [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]  -  0.12693054534140227
505  -  [2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]  -  0.12693054534140227
506  -  [1, 0

566  -  [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]  -  0.2145591551764051
567  -  [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]  -  0.12693054534140227
568  -  [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]  -  0.2145591551764051
569  -  [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]  -  0.12693054534140227
570  -  [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]  -  0.2145591551764051
571  -  [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]  -  0.2145591551764051
572  -  [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]  -  0.2145591551764051
573  -  [1, 0, 0, 

637  -  [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]  -  0.2145591551764051
638  -  [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]  -  0.12693054534140227
639  -  [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]  -  0.12693054534140227
640  -  [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]  -  0.12693054534140227
641  -  [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]  -  0.12693054534140227
642  -  [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]  -  0.12693054534140227
643  -  [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]  -  0.12693054534140227
644  -  [1, 0,